In [9]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
import pandas as pd
import csv

Datos=[]
Encabezado="Nombre de la Imagen,X,Y,W,H\n"
Datos.append(Encabezado)



direcciones= 'C:\\Users\\geske\\Documents\\Vision computacional\\vision_programas-master\\IMAGENES_MANZANAS_SD'
ListaCarpetas=os.listdir(direcciones)

for fichero in ListaCarpetas:
    List1=os.listdir(os.path.join(direcciones,fichero))  
    for fi2 in List1:
        List2=os.listdir(os.path.join(direcciones,fichero,fi2))
        for fi3 in List2: 
            img_color=cv2.imread(os.path.join(direcciones, fichero,fi2, fi3))
            b,g,r = cv2.split(img_color)
            img_filtrada=cv2.GaussianBlur(b,(7,7),5)
            ret3, inverse_mask = cv2.threshold(img_filtrada,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            mask = cv2.bitwise_not(inverse_mask)  
            kernel = np.ones((7,7),np.uint8)
            mask_morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
            kernel2 = np.ones((7,7),np.uint8)
            mask_morph2 = cv2.morphologyEx(mask_morph, cv2.MORPH_OPEN, kernel2,)
            char='mask_'+fi3+'.png'
            actual=os.getcwd()
            os.chdir(os.path.join(actual,'IMAGENES_MANZANAS_SD',fichero,fi2))
            cv2.imwrite(char,mask_morph2)
            contorno,jerarquia=cv2.findContours(mask_morph,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            areas = [cv2.contourArea(c) for c in contorno]
            max_index = np.argmax(areas)
            cnt=contorno[max_index]
            x,y,w,h=cv2.boundingRect(cnt)
            crop_img = img_color[y:y+h, x:x+w,:]
            char2='Recorte'+fi3+'.png'
            cv2.imwrite(char2,crop_img)
            os.chdir(actual)
            DatoOb="\\"+str(fichero)+"\\"+str(fi2)+"\\"+str(fi3)+","+str(x)+","+str(y)+","+str(w)+","+str(h)+"\n"
            Datos.append(DatoOb)
file=open("ManzaNaData.csv","w")
file.writelines(Datos) 
file.close()

In [4]:
import cv2
img_color=cv2.imread('C:\\Users\\geske\\Documents\\Vision computacional\\vision_programas-master\\IMAGENES_MANZANAS_SD\\test\\Gala_d\\ManzanaV_2009.jpg')
b,g,r = cv2.split(img_color)
g_filtrada=cv2.GaussianBlur(b,(7,7),5)
cv2.imwrite("imageGauss.png",g_filtrada)
ret3, inverse_mask = cv2.threshold(g_filtrada,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
mask = cv2.bitwise_not(inverse_mask)  
cv2.imwrite("OtsuUmb.png",mask)


True

In [5]:
import numpy as np
kernel = np.ones((7,7),np.uint8)
mask_morph = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
kernel2 = np.ones((7,7),np.uint8)
mask_morph2 = cv2.morphologyEx(mask_morph, cv2.MORPH_OPEN, kernel2,)
cv2.imwrite("OperMorf.png",mask_morph2)

True

In [8]:
from skimage import measure 
labels = measure.label(mask_morph, connectivity=2, background=0)
mask = np.zeros(mask_morph2.shape, dtype='uint8')
print(f'Se encontraron {len(np.unique(labels))} regiones.')

# Ciclamos sobre cada etiqueta.
for i, label in enumerate(np.unique(labels)):
    if label == 0:
        print('Etiqueta: 0 (fondo)')
        continue

    # Construimos una máscara para mostrar sólo las componentes conectadas de la etiqueta actual.
    print(f'Etiqueta: {i}')
    label_mask = np.zeros(mask_morph2.shape, dtype='uint8')
    label_mask[labels == label] = 255
    num_pixels = cv2.countNonZero(label_mask)

    # Si el número de píxeles en la componente es suficientemente larga, probablemente sea una región de interés,
    # por lo que la agregamos a nuestra máscara final.
    if 300 < num_pixels < 1500:
        mask = cv2.add(mask, label_mask)
    
        # Mostramos la máscara de *esta* etiqueta.
    cv2.imshow('Etiqueta', label_mask)
    cv2.waitKey(0)

# Mostramos las componentes más grandes (es decir, regiones de interés).
cv2.imshow('Regiones de interés', mask)
cv2.waitKey(0)


Se encontraron 2 regiones.
Etiqueta: 0 (fondo)
Etiqueta: 1


-1